# TODO：概要

# ライブラリのインポート

In [1]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

In [2]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

In [3]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# ユーティリティ関数

In [4]:
def agent_run_outputs(thread_id, project_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = project_client.agents.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                project_client.agents.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

# ツールの定義

## カスタム関数

In [5]:
# 主要な都道府県庁所在地の緯度・経度データ
PREFECTURE_LOCATIONS = {
    "北海道": (43.06417, 141.34694),   # 札幌市
    "青森県": (40.82444, 140.74),      # 青森市
    "岩手県": (39.70361, 141.1525),   # 盛岡市
    "宮城県": (38.26889, 140.87194),  # 仙台市
    "秋田県": (39.71861, 140.1025),   # 秋田市
    "山形県": (38.24056, 140.36333),  # 山形市
    "福島県": (37.75, 140.46778),     # 福島市
    "茨城県": (36.34139, 140.44667),  # 水戸市
    "栃木県": (36.56583, 139.88361),  # 宇都宮市
    "群馬県": (36.39111, 139.06083),  # 前橋市
    "埼玉県": (35.85694, 139.64889),  # さいたま市
    "千葉県": (35.60472, 140.12333),  # 千葉市
    "東京都": (35.68944, 139.69167),  # 新宿区
    "神奈川県": (35.44778, 139.6425), # 横浜市
    "新潟県": (37.90222, 139.02361),  # 新潟市
    "富山県": (36.69528, 137.21139),  # 富山市
    "石川県": (36.59444, 136.62556),  # 金沢市
    "福井県": (36.06528, 136.22194),  # 福井市
    "山梨県": (35.66389, 138.56833),  # 甲府市
    "長野県": (36.65139, 138.18111),  # 長野市
    "岐阜県": (35.39111, 136.72222),  # 岐阜市
    "静岡県": (34.97694, 138.38306),  # 静岡市
    "愛知県": (35.18028, 136.90667),  # 名古屋市
    "三重県": (34.73028, 136.50861),  # 津市
    "滋賀県": (35.00444, 135.86833),  # 大津市
    "京都府": (35.02139, 135.75556),  # 京都市
    "大阪府": (34.68639, 135.52),     # 大阪市
    "兵庫県": (34.69139, 135.18306),  # 神戸市
    "奈良県": (34.68528, 135.83278),  # 奈良市
    "和歌山県": (34.22611, 135.1675), # 和歌山市
    "鳥取県": (35.50361, 134.23833),  # 鳥取市
    "島根県": (35.47222, 133.05056),  # 松江市
    "岡山県": (34.66167, 133.935),    # 岡山市
    "広島県": (34.39639, 132.45944),  # 広島市
    "山口県": (34.18583, 131.47139),  # 山口市
    "徳島県": (34.06583, 134.55944),  # 徳島市
    "香川県": (34.34028, 134.04333),  # 高松市
    "愛媛県": (33.84167, 132.76611),  # 松山市
    "高知県": (33.55972, 133.53111),  # 高知市
    "福岡県": (33.60639, 130.41806),  # 福岡市
    "佐賀県": (33.24944, 130.29889),  # 佐賀市
    "長崎県": (32.74472, 129.87361),  # 長崎市
    "熊本県": (32.78972, 130.74167),  # 熊本市
    "大分県": (33.23806, 131.6125),   # 大分市
    "宮崎県": (31.91111, 131.42389),  # 宮崎市
    "鹿児島県": (31.56028, 130.55806),# 鹿児島市
    "沖縄県": (26.2125, 127.68111),   # 那覇市
}


def get_prefecture_location(prefecture: str) -> str:
    """
    都道府県名から緯度・経度を取得します。都道府県一覧・座標は変更しません。

    :param prefecture (str): 都道府県名（例: "東京都"）
    :rtype: str

    :return: {"latitude": ..., "longitude": ...} 形式のJSON文字列。見つからなければ {"latitude": null, "longitude": null}
    :rtype: str
    """
    loc = PREFECTURE_LOCATIONS.get(prefecture)
    if loc:
        return json.dumps({"latitude": loc[0], "longitude": loc[1]})
    else:
        return json.dumps({"latitude": None, "longitude": None})


def get_current_time_jst() -> str:
    """
    日本標準時（JST, UTC+9）の今日の日付を "YYYY-MM-DD" 形式の文字列で返します。日時情報は変更しません。

    :rtype: str

    :return: 今日の日付（"YYYY-MM-DD" 形式の文字列）
    :rtype: str
    """
    jst = zoneinfo.ZoneInfo("Asia/Tokyo")
    now = datetime.datetime.now(jst)
    return now.strftime("%Y-%m-%d")


def get_temperature(latitude: float, longitude: float, date: str) -> str:
    """
    指定した緯度・経度と日付に基づき、Open-Meteo APIで該当日の平均気温（摂氏）を返します。
    外部サービスのみ参照し、情報は変更しません。

    :param latitude (float): 緯度
    :param longitude (float): 経度
    :param date (str): 日付（"YYYY-MM-DD" 形式）
    :rtype: str

    :return: 指定日の平均気温（摂氏）をJSON文字列で返します。データがなければ {"temperature": null}。
    :rtype: str
    """
    url = (
        "https://api.open-meteo.com/v1/forecast"
        f"?latitude={latitude}&longitude={longitude}"
        f"&start_date={date}&end_date={date}"
        "&daily=temperature_2m_mean"
        "&timezone=Asia%2FTokyo"
    )
    resp = requests.get(url, timeout=5)
    resp.raise_for_status()
    data = resp.json()
    temps = data.get("daily", {}).get("temperature_2m_mean", [])
    value = temps[0] if temps else None
    return json.dumps({"temperature": value})


## 関数のテスト

In [6]:
prefecture_name = "北海道"

# 都道府県名から緯度・経度を取得
loc_json = get_prefecture_location(prefecture_name)

# 緯度・経度を取得
loc = json.loads(loc_json)
lat = loc["latitude"]
lon = loc["longitude"]

# 現在の日付を取得
today = get_current_time_jst()

# 平均気温を取得
temp_json = get_temperature(lat, lon, today)
temp = json.loads(temp_json)["temperature"]

print(f"{today} の {prefecture_name} の平均気温: {temp}°C")


2025-07-27 の 北海道 の平均気温: 24.0°C


## Toolset に格納

In [7]:
# Toolset の作成＆関数の追加
toolset = ToolSet()

functions = FunctionTool(functions={get_prefecture_location, get_current_time_jst, get_temperature})
toolset.add(functions)

# エージェントの作成

In [8]:
custom_functions_agent = project_client.agents.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="custom_functions_agent",

    # instructions:
    #  - エージェントの「システム指示文」を設定し、エージェントのふるまいなどを制御できる
    instructions=(
        "あなたは日本国内の気温や地理情報に答えるアシスタントです。"
        "利用可能なツールを確認し、必要に応じて使用してください。"        
    ),

    # description:
    #  - エージェントの説明を設定し、ユーザーや他のエージェントがこのエージェントの目的を理解するための情報を提供
    description=(
        "日本の都道府県の地理情報（緯度・経度）や、日本標準時の日付、指定した場所・日付の気温データを自動的に取得し、"
        "ユーザーの質問に答えるアシスタントです。 "
    ),

    # tools:
    #  - エージェントが使用できるツールのセットを指定
    tools=toolset.definitions,
)
print(f"Created Agent. AGENT_ID: {custom_functions_agent.id}")


Created Agent. AGENT_ID: asst_am1YRL5LeWPO1cMYZlvkLeum


In [9]:
# カスタム関数の自動呼出しを有効化
project_client.agents.enable_auto_function_calls(toolset)

In [10]:
# agent_dict = agent.as_dict()
# print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

# スレッドの作成

In [11]:
# Thread の作成
thread = project_client.agents.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_opMILYZb641eJbfjuZrYKQKi


# ユーザーメッセージの追加

In [12]:
# メッセージの追加
user_message = "Tokyoの今日の気温を教えて。"
# user_message = "過去３日間の沖縄県の平均気温を教えて。"

message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_l7sIYK5aOf0MCvTjzyuvoYvt


# Run の実行

In [13]:
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=custom_functions_agent.id
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, project_client)

USER: Tokyoの今日の気温を教えて。
ASSISTANT: 東京（Tokyo）の今日（2025年7月27日）の平均気温は約28.4℃です。


※ ここで、Azure AI Foundry 上の Web UI からエージェントのトレースを確認してみよう。

# Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [14]:
# 変数の定義
agent_env_key = "FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID"
agent_env_value = custom_functions_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')

.envファイルに FOUNDRY_CUSTOM_FUNCTIONS_AGENT_ID="asst_am1YRL5LeWPO1cMYZlvkLeum" を追記しました。


# MCP 対応

In [15]:
import time
import json

from azure.ai.agents.models import MessageTextContent, ListSortOrder
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import McpTool, MCPToolDefinition, RequiredMcpToolCall, SubmitToolApprovalAction, ToolApproval

In [18]:
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)

with project_client:    
    agent = project_client.agents.create_agent(
        model="gpt-4.1",
        name="my-mcp-agent",
        instructions=(
            "You are a helpful assistant. "
            "Use the tools provided to answer the user's questions. "
            "Be sure to cite your sources."
        ),
        tools=mcp_tool.definitions
    )
    print(f"Created agent, ID: {agent.id}")
    print(agent.as_dict())

    thread = project_client.agents.threads.create()
    print(f"Created thread, ID: {thread.id}")

    message = project_client.agents.messages.create(
        thread_id=thread.id,
        role="user",
        content="Azure AI Foundry Agent Service の最新情報は？"
    )
    print(f"Created message, ID: {message.id}")

    
    run = project_client.agents.runs.create(
        thread_id=thread.id,
        agent_id=agent.id
    )


    while run.status in ["queued", "in_progress", "requires_action"]:
        time.sleep(1)
        run = project_client.agents.runs.get(thread_id=thread.id, run_id=run.id)

        if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
            tool_calls = run.required_action.submit_tool_approval.tool_calls
            if not tool_calls:
                print("No tool calls provided - cancelling run")
                project_client.agents.runs.cancel(thread_id=thread.id, run_id=run.id)
                break

            tool_approvals = []
            for tool_call in tool_calls:
                if isinstance(tool_call, RequiredMcpToolCall):
                    try:
                        print(f"Approving tool call: {tool_call}")
                        tool_approvals.append(
                            ToolApproval(
                                tool_call_id=tool_call.id,
                                approve=True,
                                headers=mcp_tool.headers,
                            )
                        )
                    except Exception as e:
                        print(f"Error approving tool_call {tool_call.id}: {e}")

            print(f"tool_approvals: {tool_approvals}")
            if tool_approvals:
                project_client.agents.runs.submit_tool_outputs(
                    thread_id=thread.id,
                    run_id=run.id,
                    tool_approvals=tool_approvals
                )

        print(f"Current run status: {run.status}")

    # Retrieve the generated response:
    messages = project_client.agents.messages.list(thread_id=thread.id)
    print("\nConversation:")
    print("-" * 50)

    for msg in messages:
        if msg.text_messages:
            last_text = msg.text_messages[-1]
            print(f"{msg.role.upper()}: {last_text.text.value}")
            print("-" * 50)

    # project_client.agents.delete_agent(agent.id)
    # print("Deleted agent")


Created agent, ID: asst_nrqxSZAQldlhrUEgtn6YttOK
{'id': 'asst_nrqxSZAQldlhrUEgtn6YttOK', 'object': 'assistant', 'created_at': 1753555238, 'name': 'my-mcp-agent', 'description': None, 'model': 'gpt-4.1', 'instructions': "You are a helpful assistant. Use the tools provided to answer the user's questions. Be sure to cite your sources.", 'tools': [{'type': 'mcp', 'server_label': 'MicrosoftDocs', 'server_url': 'https://learn.microsoft.com/api/mcp', 'allowed_tools': None}], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'}
Created thread, ID: thread_0UAWQmJjAht71X13lurxzzTV
Created message, ID: msg_JocTiWYIUDG9TiSiN0z5IMWz
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Approving tool call: {'id': 'call_hWeKEFphLxutj2tleOcL8um0', 'type': 'mcp', 'arguments': '{"question":"Azure AI Foundry Agent Service æ\x9c\x80æ\x96°æ\x83\x85å\xa0±"}', 'name': 'microsoft_docs_search', 'server_label': 'MicrosoftDocs'}
tool_ap